In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [5]:
# Load data
df = pd.read_csv('../data/Imputing Rationales.csv')

# Basic overview
print(f"Total observations: {len(df)}")
print(f"Columns: {df.columns.tolist()}")
print(f"Memory usage: {df.memory_usage(deep=True).sum() / 1e6:.2f} MB")

df.head()

Total observations: 3470442
Columns: ['investor_id', 'ProxySeason', 'pid', 'meeting_id', 'N_Meetings_Dir', 'N_Meetings_Rat', 'frac_vote_against', 'ind_dissent', 'boardstructure', 'diversity', 'indep', 'tenure', 'governance', 'combined_ceo_chairman', 'esg_csr', 'responsiveness', 'attendance', 'compensation', 'busyness', 'norat_misc', 'European', 'North_America', 'ROW', 'Country', 'US', 'Robo_Voter_ISS', 'Robo_Voter_GL', 'Robo_Voter', 'ProMgmt', 'Pension', 'Fund_manager', 'Other_investors', 'N_Meetings', 'N_dissent', 'Per_Independent', 'AvTenure', 'AvBusy', 'CEO_Duality', 'Per_female', 'D_Per_female', 'D_AvTenure', 'D_Independent', 'D_Busy', 'E_Index', 'E_Index_nomissing', 'Dissent', 'Contentious_ISS', 'Contentious_GL', 'ln_MktCap', 'ROA', 'Mkt_to_Book', 'Dividends', 'Lev', 'InstOwn_Perc', 'ffi', 'AdjEBITDA', 'alpha_mm', 'alpha_4f', 'TSR', 'boardstructure_tc', 'diversity_tc', 'indep_tc', 'tenure_tc', 'governance_tc', 'combined_ceo_chairman_tc', 'esg_csr_tc', 'responsiveness_tc', 'attenda

,investor_id,ProxySeason,pid,meeting_id,N_Meetings_Dir,N_Meetings_Rat,frac_vote_against,ind_dissent,boardstructure,diversity,...,diversity_tc,indep_tc,tenure_tc,governance_tc,combined_ceo_chairman_tc,esg_csr_tc,responsiveness_tc,attendance_tc,compensation_tc,busyness_tc
0,2,2014,44,84036,1377,356,0.083333,1,0.0,0.0,...,0.0,0.133333,0.000000,0.066667,0.200000,0.0,0.066667,0.066667,0.466667,0.000000
1,2,2014,184,80726,1377,356,0.181818,1,0.0,0.0,...,0.0,0.285714,0.285714,0.000000,0.285714,0.0,0.000000,0.000000,0.000000,0.142857
2,2,2014,168,82137,1377,356,0.250000,1,0.0,0.0,...,0.0,0.266667,0.200000,0.066667,0.333333,0.0,0.000000,0.000000,0.000000,0.000000
3,2,2014,32,82100,1377,356,0.090909,1,0.0,0.0,...,0.0,0.272727,0.090909,0.090909,0.363636,0.0,0.000000,0.000000,0.090909,0.090909
4,2,2014,37,82101,1377,356,0.153846,1,0.0,0.0,...,0.0,0.363636,0.090909,0.181818,0.272727,0.0,0.000000,0.000000,0.000000,0.090909


In [9]:
# Quick sanity checks
print("1. Data shape:", df.shape)
print("\n2. Key columns exist?")
for col in ['investor_id', 'ProxySeason', 'ind_dissent', 'diversity', 'tenure']:
    print(f"  {col}: {col in df.columns}")

print("\n3. Dissent votes:", df['ind_dissent'].sum())
print("   Total votes:", len(df))

print("\n4. Labeled rationales:")
rationale_cols = ['diversity', 'indep', 'tenure', 'busyness']
for col in rationale_cols:
    if col in df.columns:
        labeled = df[col].notna().sum()
        print(f"  {col}: {labeled}")

print("\n5. Unique investors:", df['investor_id'].nunique())
print("   Unique meetings:", df['meeting_id'].nunique() if 'meeting_id' in df.columns else 'N/A')

1. Data shape: (3470442, 71)

2. Key columns exist?
  investor_id: True
  ProxySeason: True
  ind_dissent: True
  diversity: True
  tenure: True

3. Dissent votes: 993401
   Total votes: 3470442

4. Labeled rationales:
  diversity: 162797
  indep: 162797
  tenure: 162797
  busyness: 162797

5. Unique investors: 1603
   Unique meetings: 33781


In [7]:
# 1. DISSENT DISTRIBUTION
print("=== Dissent Distribution ===")
print(f"Total dissenting votes (ind_dissent=1): {df['ind_dissent'].sum()}")
print(f"Dissent rate: {df['ind_dissent'].mean():.2%}")

# 2. RATIONALE DISCLOSURE PATTERNS
print("\n=== Rationale Disclosure ===")
# Check which observations have ANY rationale disclosed
rationale_cols = ['diversity', 'indep', 'tenure', 'busyness', 
                  'combined_ceo_chairman', 'boardstructure', 
                  'governance', 'esg_csr', 'responsiveness', 
                  'attendance', 'compensation', 'norat_misc']

# Find rows with at least one rationale
df['has_rationale'] = df[rationale_cols].notna().any(axis=1)
print(f"Observations with rationale: {df['has_rationale'].sum()}")
print(f"Disclosure rate: {df['has_rationale'].mean():.2%}")

# Among dissenting votes
dissent_df = df[df['ind_dissent'] == 1]
print(f"\nAmong dissenting votes:")
print(f"  With rationale: {dissent_df['has_rationale'].sum()}")
print(f"  Without rationale: {(~dissent_df['has_rationale']).sum()}")

# 3. INVESTOR-LEVEL ANALYSIS
print("\n=== Investor Patterns ===")
investor_stats = df.groupby('investor_id').agg({
    'meeting_id': 'count',  # Total meetings
    'ind_dissent': 'sum',    # N_dissent
    'has_rationale': 'sum'   # N_Meetings_Rat
}).rename(columns={
    'meeting_id': 'N_Meetings',
    'ind_dissent': 'N_dissent',
    'has_rationale': 'N_Meetings_Rat'
})

print(f"Total investors: {len(investor_stats)}")
print(f"Investors with 0 rationales (to drop): {(investor_stats['N_Meetings_Rat'] == 0).sum()}")
print("\nInvestor statistics:")
print(investor_stats.describe())

# 4. LABEL DISTRIBUTION (Key for understanding class imbalance)
print("\n=== Label Frequencies (among disclosed rationales) ===")
for col in ['diversity', 'indep', 'tenure', 'busyness', 'combined_ceo_chairman']:
    if col in df.columns:
        labeled = df[df[col].notna()]
        if len(labeled) > 0:
            positive_rate = labeled[col].mean()
            print(f"{col:20s}: {positive_rate:.2%} (n={len(labeled)})")

# 5. TEMPORAL DISTRIBUTION
print("\n=== Temporal Coverage ===")
print(df.groupby('ProxySeason')['meeting_id'].count().sort_index())

=== Dissent Distribution ===
Total dissenting votes (ind_dissent=1): 993401
Dissent rate: 28.62%

=== Rationale Disclosure ===
Observations with rationale: 162797
Disclosure rate: 4.69%

Among dissenting votes:
  With rationale: 162797
  Without rationale: 830604

=== Investor Patterns ===
Total investors: 1603
Investors with 0 rationales (to drop): 1363

Investor statistics:
         N_Meetings     N_dissent  N_Meetings_Rat
count   1603.000000   1603.000000     1603.000000
mean    2164.966937    619.713662      101.557704
std     4465.287715   1667.533641      714.461026
min        1.000000      0.000000        0.000000
25%      156.500000      7.000000        0.000000
50%      465.000000     55.000000        0.000000
75%     1688.500000    325.500000        0.000000
max    31442.000000  18034.000000    13991.000000

=== Label Frequencies (among disclosed rationales) ===
diversity           : 31.06% (n=162797)
indep               : 37.19% (n=162797)
tenure              : 20.73% (n=162

In [8]:
# Create filtered dataset based on requirements
print("\n=== Applying Filters ===")
print(f"Starting observations: {len(df)}")

# Filter 1: Drop non-disclosers (N_Meetings_Rat = 0)
df_filtered = df.merge(investor_stats[['N_Meetings_Rat']], 
                       left_on='investor_id', 
                       right_index=True)
df_filtered = df_filtered[df_filtered['N_Meetings_Rat'] > 0]
print(f"After dropping non-disclosers: {len(df_filtered)}")

# Filter 2: Try minimum rationale threshold (start with 5)
MIN_RAT = 5
df_filtered = df_filtered[df_filtered['N_Meetings_Rat'] >= MIN_RAT]
print(f"After N_Meetings_Rat >= {MIN_RAT}: {len(df_filtered)}")

# Filter 3: Try minimum dissent threshold
df_filtered = df_filtered.merge(investor_stats[['N_dissent']], 
                                 left_on='investor_id', 
                                 right_index=True)
MIN_DISSENT = 5
df_filtered = df_filtered[df_filtered['N_dissent'] >= MIN_DISSENT]
print(f"After N_dissent >= {MIN_DISSENT}: {len(df_filtered)}")

# Focus on dissenting votes only (ind_dissent = 1)
df_dissent = df_filtered[df_filtered['ind_dissent'] == 1].copy()
print(f"Final dissenting votes for modeling: {len(df_dissent)}")

# Check train/test potential
print(f"\nLabeled (for training): {df_dissent['has_rationale'].sum()}")
print(f"Unlabeled (to predict): {(~df_dissent['has_rationale']).sum()}")


=== Applying Filters ===
Starting observations: 3470442


KeyError: 'N_Meetings_Rat'